In [2]:
from tensorflow.keras import backend, layers, metrics

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.utils import plot_model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


import os
import cv2
import time
import random
import numpy as np

import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import preprocess_input

import seaborn as sns
import matplotlib.pyplot as plt
tf.__version__, np.__version__

import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, BatchNormalization, Dropout,Flatten ,LeakyReLU
from tensorflow.keras.initializers import GlorotNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping , LearningRateScheduler ,ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.regularizers import l2
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import cv2
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier

from skimage import feature, filters, io, color
from skimage.transform import resize
from skimage.feature import canny
from skimage.filters import sobel
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
import os
import time
import random
import numpy as np
import pandas as pd
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import backend, layers, metrics
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import (
    Input, Conv2D, UpSampling2D, Concatenate, Dense, GlobalAveragePooling2D,
    MaxPooling2D, BatchNormalization, Dropout, Flatten, LeakyReLU
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import (
    ResNet50, VGG16, NASNetMobile, MobileNetV2, InceptionV3,
    DenseNet121, Xception, EfficientNetB0
)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import (
    ImageDataGenerator, load_img, img_to_array
)
from tensorflow.keras.utils import (
    Sequence, to_categorical, plot_model
)
from tensorflow.keras.callbacks import (
    EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
)
from tensorflow.keras.initializers import GlorotNormal
from tensorflow.keras.regularizers import l2
tf.__version__, np.__version__

('2.6.2', '1.20.3')

#### **Load Image** 

In [3]:
def load_images_and_create_map(directory, plant_type_mapping):
    img_label_map = {}

    label_encoder = LabelEncoder()
    label_encoder.fit(list(plant_type_mapping.keys()))


    for plant_type, disease_folders in plant_type_mapping.items():

        for disease_folder in disease_folders:
            disease_folder_path = os.path.join(directory, disease_folder)

            for img_name in os.listdir(disease_folder_path):
                img_path = os.path.join(disease_folder_path, img_name)

                if os.path.isfile(img_path):

                    label = label_encoder.transform([plant_type])[0]

                    img_label_map[img_path] = label

    return img_label_map



def preprocess_image(image_path):
    img = load_img(image_path, target_size=(256, 256))
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    return img_array

In [4]:
train_dir = '/kaggle/input/data-vision2/Project Data/Train/images'
val_dir = '/kaggle/input/data-vision2/Project Data/Val/images/'

In [5]:
plant_type_mapping = {
    "Apple": ["Apple___Apple_scab", "Apple___Black_rot", "Apple___Cedar_apple_rust", "Apple___healthy"],
    "Cherry_(including_sour)": ["Cherry_(including_sour)___healthy", "Cherry_(including_sour)___Powdery_mildew"],
    "Corn_(maize)": ["Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot", "Corn_(maize)___Common_rust_", "Corn_(maize)___healthy", "Corn_(maize)___Northern_Leaf_Blight"],
    "Grape": ["Grape___Black_rot", "Grape___Esca_(Black_Measles)", "Grape___healthy", "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)"],
    "Peach": ["Peach___Bacterial_spot", "Peach___healthy"],
    "Pepper,_bell": ["Pepper,_bell___Bacterial_spot", "Pepper,_bell___healthy"],
    "Potato": ["Potato___Early_blight", "Potato___healthy", "Potato___Late_blight"],
    "Strawberry": ["Strawberry___healthy", "Strawberry___Leaf_scorch"],
    "Tomato": ["Tomato___Bacterial_spot", "Tomato___Early_blight", "Tomato___healthy", "Tomato___Late_blight",
               "Tomato___Leaf_Mold", "Tomato___Septoria_leaf_spot", "Tomato___Spider_mites Two-spotted_spider_mite",
               "Tomato___Target_Spot", "Tomato___Tomato_mosaic_virus", "Tomato___Tomato_Yellow_Leaf_Curl_Virus"]
}

In [ ]:
train_img_label_map = load_images_and_create_map(train_dir, plant_type_mapping)
print("Training dataset mapping length:", len(train_img_label_map))

val_img_label_map = load_images_and_create_map(val_dir, plant_type_mapping)
print("Validation dataset mapping length:", len(val_img_label_map))


In [ ]:
train_images = np.array([preprocess_image(img_path) for img_path in train_img_label_map.keys()])
train_labels = np.array(list(train_img_label_map.values()))

val_images = np.array([preprocess_image(img_path) for img_path in val_img_label_map.keys()])
val_labels = np.array(list(val_img_label_map.values()))

train_labels = to_categorical(train_labels, num_classes=9)
val_labels = to_categorical(val_labels, num_classes=9)


In [ ]:
print("Train Images Shape " ,train_images.shape )

print("Val Images Shape" ,val_images.shape )

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_datagen.fit(train_images)
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(train_images, train_labels, batch_size=32)
val_generator = val_datagen.flow(val_images, val_labels, batch_size=32)


## **Models** 

## **DenseNet** 

In [ ]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False

model_densenet = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(9, activation='softmax')
])

model_densenet.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Measure training time
start_time = time.time()
history = model_densenet.fit(train_generator, validation_data=val_generator, epochs=10, verbose=1)
end_time = time.time()
training_time = end_time - start_time
print(f"\nTotal Training Time: {training_time:.2f} seconds")

print("\nFinal Training and Validation Accuracy for DenseNet121:")
for epoch, (train_acc, val_acc) in enumerate(zip(history.history['accuracy'], history.history['val_accuracy']), 1):
    print(f"Epoch {epoch}: Training Accuracy = {train_acc:.4f}, Validation Accuracy = {val_acc:.4f}")
# Measure testing time
start_time = time.time()
train_loss, train_acc = model_densenet.evaluate(train_generator)
end_time = time.time()
testing_time = end_time - start_time
print(f"\nTotal Testing Time: {testing_time:.2f} seconds")



In [ ]:
val_loss, val_acc = model_densenet.evaluate(val_generator)
print(f"\nFinal Training Accuracy: {train_acc:.4f}, Final Validation Accuracy: {val_acc:.4f}")

In [ ]:
model_densenet.save("model_densenet.h5")

## **MobileNetV2**

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

base_model.trainable = False

model_mobile = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(9, activation='softmax')
])

model_mobile.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

start_time = time.time()
history = model_mobile.fit(train_generator, validation_data=val_generator, epochs=10)
end_time = time.time()
training_time = end_time - start_time
print(f"\nTotal Training Time: {training_time:.2f} seconds")


print("\nFinal Training and Validation Accuracy for MobileNetV2:")
for epoch, (train_acc, val_acc) in enumerate(zip(history.history['accuracy'], history.history['val_accuracy']), 1):
    print(f"Epoch {epoch}: Training Accuracy = {train_acc:.4f}, Validation Accuracy = {val_acc:.4f}")
    
start_time = time.time()
train_loss, train_acc = model_mobile.evaluate(train_generator)
end_time = time.time()
training_time = end_time - start_time
print(f"\nTotal Testing Time: {training_time:.2f} seconds")


In [ ]:
val_loss, val_acc = model_mobile.evaluate(val_generator)
print(f"\nFinal Training Accuracy: {train_acc:.4f}, Final Validation Accuracy: {val_acc:.4f}")

In [ ]:
model_mobile.save("model_mobile.h5")

### **MobileNetV2 Part 2**

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    elif epoch < 20:
        return lr * 0.5
    else:
        return lr * 0.1

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False  

model_mobile_layers = Sequential()
model_mobile_layers.add(base_model)  

model_mobile_layers.add(Conv2D(32, (5, 5), activation=None, padding='same'))
model_mobile_layers.add(LeakyReLU(alpha=0.1))
model_mobile_layers.add(BatchNormalization())

model_mobile_layers.add(MaxPooling2D(pool_size=(2, 2)))

model_mobile_layers.add(Conv2D(64, (5, 5), activation=None, padding='same'))
model_mobile_layers.add(LeakyReLU(alpha=0.1))
model_mobile_layers.add(BatchNormalization())

model_mobile_layers.add(MaxPooling2D(pool_size=(2, 2)))

model_mobile_layers.add(Conv2D(128, (3, 3), activation=None, padding='same'))
model_mobile_layers.add(LeakyReLU(alpha=0.1))
model_mobile_layers.add(BatchNormalization())

model_mobile_layers.add(GlobalAveragePooling2D())

model_mobile_layers.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
model_mobile_layers.add(Dropout(0.3))
model_mobile_layers.add(Dense(9, activation='softmax'))  

model_mobile_layers.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_mobile_layers.summary()

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

lr_callback = LearningRateScheduler(scheduler)

start_time = time.time()

history = model_mobile_layers.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stopping, lr_callback]
)

end_time = time.time()
training_time = end_time - start_time
print(f"\nTotal Training Time: {training_time:.2f} seconds")

start_time = time.time()
val_loss, val_acc = model_mobile_layers.evaluate(val_generator)
end_time = time.time()
training_time = end_time - start_time
print(f"\nTotal Testing Time: {training_time:.2f} seconds")
print(f"Final Validation Accuracy: {val_acc}")



In [ ]:
model_mobile_layers.save("model_mobile_layers.h5")

In [ ]:
### BSET MODEL 
model = model_mobile_layers 

## **HOG Feature Extraction**

In [ ]:
def getfeatures(image_path):
    image = imread(image_path, as_gray=True)
    # image = imread(image_path)
    image_resized = resize(image, (128, 64))
    fd, _ = hog(image_resized, orientations=16, pixels_per_cell=(16, 16),
                cells_per_block=(2, 2), visualize=True)
    return fd


In [ ]:
X_train = np.array([getfeatures(img_path) for img_path in train_img_label_map.keys()])
y_train = np.array(list(train_img_label_map.values()))

X_val = np.array([getfeatures(img_path) for img_path in val_img_label_map.keys()])
y_val = np.array(list(val_img_label_map.values()))


## **Linear**

In [ ]:
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)

start_time = time.time()
y_pred_linear = svm_linear.predict(X_train)
accuracy_linear = accuracy_score(y_train, y_pred_linear)
end_time = time.time()
training_time = end_time - start_time
print(f"\nTotal Training Time: {training_time:.2f} seconds")
print("Training Accuracy using Linear Kernel:", accuracy_linear)

start_time = time.time()
y_pred_linear = svm_linear.predict(X_val)
accuracy_linear = accuracy_score(y_val, y_pred_linear)
print("Testing Accuracy using Linear Kernel:", accuracy_linear)
end_time = time.time()
training_time = end_time - start_time
print(f"\nTotal Testing Time: {training_time:.2f} seconds")

plant_type_names = sorted(list(plant_type_mapping.keys()))
unique_labels = list(range(len(plant_type_names)))
class_names = [plant_type_names[label] for label in unique_labels]

cm_linear = confusion_matrix(y_val, y_pred_linear, labels=unique_labels)
df_cm_linear = pd.DataFrame(cm_linear, index=class_names, columns=class_names)

plt.figure(figsize=(12, 8))
sns.heatmap(df_cm_linear, annot=True, fmt='d', cmap='Blues', linewidths=0.5)
plt.title("Confusion Matrix - Linear Kernel")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()



## **RBF**

In [ ]:

svm_rbf = SVC(kernel='rbf', gamma=0.1)
start_time = time.time()
svm_rbf.fit(X_train, y_train)
y_train_pred_rbf = svm_rbf.predict(X_train)

# Compute training accuracy
accuracy_train_rbf = accuracy_score(y_train, y_train_pred_rbf)
end_time = time.time()
training_time = end_time - start_time
print(f"\nTotal Training Time: {training_time:.2f} seconds")
print("Training Accuracy using RBF Kernel:", accuracy_train_rbf)

start_time = time.time()
y_pred_rbf = svm_rbf.predict(X_val)
accuracy_rbf = accuracy_score(y_val, y_pred_rbf)
print("Accuracy using RBF Kernel:", accuracy_rbf)
end_time = time.time()
training_time = end_time - start_time
print(f"\nTotal Testing Time: {training_time:.2f} seconds")

cm_rbf = confusion_matrix(y_val, y_pred_rbf, labels=unique_labels)
df_cm_rbf = pd.DataFrame(cm_rbf, index=class_names, columns=class_names)

plt.figure(figsize=(12, 8))
sns.heatmap(df_cm_rbf, annot=True, fmt='d', cmap='Greens', linewidths=0.5)
plt.title("Confusion Matrix - RBF Kernel")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


## **Reading the Dataset**


In [6]:
random.seed(5)
np.random.seed(5)
tf.random.set_seed(5)


def read_image(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

## **Create Path** 

In [7]:


def create_paths(directory):
  image_paths = {}
  for plant_type in plant_type_mapping.values():
    for plant_disease in plant_type:
      folder_path = os.path.join(directory, plant_disease)
      image_paths[plant_disease] = [
        os.path.join(folder_path, file)
        for file in os.listdir(folder_path)
        if os.path.isfile(os.path.join(folder_path, file))
      ]

  return image_paths


In [8]:
train_image_paths = create_paths(train_dir)
val_image_paths = create_paths(val_dir)


## **Create Triplets** 

In [9]:
def create_triplets(image_paths, max_files=10):
    triplets = []

    cnt = 0
    for plant_key, plant_type in plant_type_mapping.items():
      for plant_disease in plant_type:
        num_files = len(image_paths[plant_disease])
        for i in range(num_files-1):
            for j in range(i+1, num_files):
                anchor = image_paths[plant_disease][i]
                positive = image_paths[plant_disease][j]
                if cnt >= 7:
                    neg_folder = plant_disease
                    while neg_folder == plant_disease or plant_key == neg_folder_key:
                        neg_folder_key = random.choice(list(plant_type_mapping.keys()))  # Randomly choose a key
                        neg_folder = random.choice(plant_type_mapping[neg_folder_key])
                    neg_idx = random.randint(0, min(max_files, len(image_paths[neg_folder])) - 1)
                    negative = image_paths[neg_folder][neg_idx]
                    triplets.append((anchor, positive, negative))

                else:
                    neg_folder = plant_disease
                    while neg_folder == plant_disease:
                        neg_folder = random.choice(plant_type)
                    neg_idx = random.randint(0, min(max_files, len(image_paths[neg_folder])) - 1)
                    negative = image_paths[neg_folder][neg_idx]
                    triplets.append((anchor, positive, negative))
                cnt = (cnt + 1)%10

    random.shuffle(triplets)
    return triplets

train_triplet = create_triplets(train_image_paths)
val_triplet  = create_triplets(val_image_paths)


In [ ]:
import re
def get_batch2(triplet_list, batch_size=16, preprocess=True):
    batch_steps = len(triplet_list)//batch_size

    for i in range(batch_steps+1):
        anchor   = []
        positive = []
        negative = []
        
        a_names = []
        p_names = []
        n_names = []
        
        j = i*batch_size

        while j<(i+1)*batch_size and j<len(triplet_list):
            a, p, n = triplet_list[j]
            anchor.append(read_image(a))
            match = re.search(r'/images/([^/]+)/', a)
            if match:
                a_names.append(match.group(1))
            positive.append(read_image(p))
            
            match = re.search(r'/images/([^/]+)/', p)
            if match:
                p_names.append(match.group(1))
            negative.append(read_image(n))
            
            match = re.search(r'/images/([^/]+)/', n)
            if match:
                n_names.append(match.group(1))
            j+=1
            

        anchor = np.array(anchor)
        positive = np.array(positive)
        negative = np.array(negative)

        if preprocess:
            anchor = preprocess_input(anchor)
            positive = preprocess_input(positive)
            negative = preprocess_input(negative)

        
        
        
        yield ([anchor, positive, negative, a_names, p_names, n_names])


## **Creating Batch-Generator**


In [10]:
def get_batch(triplet_list, batch_size=16, preprocess=True):
    batch_steps = len(triplet_list)//batch_size

    for i in range(batch_steps+1):
        anchor   = []
        positive = []
        negative = []

        j = i*batch_size
        while j<(i+1)*batch_size and j<len(triplet_list):
            a, p, n = triplet_list[j]
            anchor.append(read_image(a))
            positive.append(read_image(p))
            negative.append(read_image(n))
            j+=1

        anchor = np.array(anchor)
        positive = np.array(positive)
        negative = np.array(negative)

        if preprocess:
            anchor = preprocess_input(anchor)
            positive = preprocess_input(positive)
            negative = preprocess_input(negative)

        yield ([anchor, positive, negative])


## **Plotting the Data**


In [ ]:
num_plots = 6

f, axes = plt.subplots(num_plots, 3, figsize=(15, 20))

for x in get_batch(train_triplet, batch_size=num_plots, preprocess=False):
    a,p,n = x
    for i in range(num_plots):
        axes[i, 0].imshow(a[i])
        axes[i, 1].imshow(p[i])
        axes[i, 2].imshow(n[i])
        i+=1
    break

In [ ]:
num_plots = 6

f, axes = plt.subplots(num_plots, 3, figsize=(15, 20))

for x in get_batch2(train_triplet, batch_size=num_plots, preprocess=False):
    a,p,n, a_name, p_name, n_name = x

    for i in range(num_plots):
        axes[i, 0].imshow(a[i])
        axes[i, 0].set_title(a_name[i])
        axes[i, 0].axis('off')  # Optional: Turn off axes

        axes[i, 1].imshow(p[i])
        axes[i, 1].set_title(p_name[i])
        axes[i, 1].axis('off')  # Optional: Turn off axes

        axes[i, 2].imshow(n[i])
        axes[i, 2].set_title(n_name[i])
        axes[i, 2].axis('off')  # Optional: Turn off axes

        i+=1
    break

In [ ]:
len(train_triplet)

## **Creating The Siamese Model**


In [11]:
def get_encoder(input_shape):
    """ Returns the image encoding model """
    pretrained_model = Xception(
        input_shape=input_shape,
        weights='imagenet',
        include_top=False,
        pooling='avg',
    )
    
    for i in range(len(pretrained_model.layers)-27):
        pretrained_model.layers[i].trainable = False

    encode_model = Sequential([
        pretrained_model,
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(256, activation="relu"),
        layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
    ], name="Encode_Model")
    return encode_model

In [12]:
class DistanceLayer(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)
    

def get_siamese_network(input_shape = (256, 256, 3)):
    encoder = get_encoder(input_shape)
    
    anchor_input   = layers.Input(input_shape, name="Anchor_Input")
    positive_input = layers.Input(input_shape, name="Positive_Input")
    negative_input = layers.Input(input_shape, name="Negative_Input")
    
    encoded_a = encoder(anchor_input)
    encoded_p = encoder(positive_input)
    encoded_n = encoder(negative_input)
    
    distances = DistanceLayer()(
        encoder(anchor_input),
        encoder(positive_input),
        encoder(negative_input)
    )
    
    siamese_network = Model(
        inputs  = [anchor_input, positive_input, negative_input],
        outputs = distances,
        name = "Siamese_Network"
    )
    return siamese_network

siamese_network = get_siamese_network()
siamese_network.summary()

83697664/83683744 [==============================] - 0s 0us/step
Model: "Siamese_Network"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Anchor_Input (InputLayer)       [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
Positive_Input (InputLayer)     [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
Negative_Input (InputLayer)     [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
Encode_Model (Sequential)       (None, 256)          22043944    Anchor_Input[0][0]               
                   

In [ ]:
plot_model(siamese_network, show_shapes=True, show_layer_names=True)

In [ ]:
class SiameseModel(Model):
    def __init__(self, siamese_network, margin=1.0):
        super(SiameseModel, self).__init__()
        
        self.margin = margin
        self.siamese_network = siamese_network
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape get the gradients when we compute loss, and uses them to update the weights
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)
            
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.siamese_network.trainable_weights))
        
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)
        
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        ap_distance, an_distance = self.siamese_network(data)
        loss = tf.maximum(ap_distance - an_distance + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        return [self.loss_tracker]

In [ ]:
siamese_model = SiameseModel(siamese_network)

optimizer = Adam(learning_rate=1e-3, epsilon=1e-01)
siamese_model.compile(optimizer=optimizer)



## **Training the Model**


In [ ]:
def test_on_triplets(test_data , batch_size = 256):
    pos_scores, neg_scores = [], []

    for data in get_batch(test_data, batch_size=batch_size):
        prediction = siamese_model.predict(data)
        pos_scores += list(prediction[0])
        neg_scores += list(prediction[1])
    
    accuracy = np.sum(np.array(pos_scores) < np.array(neg_scores)) / len(pos_scores)
    ap_mean = np.mean(pos_scores)
    an_mean = np.mean(neg_scores)
    ap_stds = np.std(pos_scores)
    an_stds = np.std(neg_scores)
    
   # print(f"Accuracy on test = {accuracy:.5f}")
    return (accuracy, ap_mean, an_mean, ap_stds, an_stds)

In [ ]:
save_all = False
epochs = 5
batch_size = 16

max_acc = 0
train_loss = []
test_metrics = []
train_metrics = []

for epoch in range(1, epochs+1):
    t = time.time()
    epoch_loss = []
    for data in get_batch(train_triplet, batch_size=batch_size):
        loss = siamese_model.train_on_batch(data)
        epoch_loss.append(loss)
    epoch_loss = sum(epoch_loss)/len(epoch_loss)
    train_loss.append(epoch_loss)

    print(f"\nEPOCH: {epoch} \t (Epoch done in {int(time.time()-t)} sec)")
    print(f"Loss on train    = {epoch_loss:.5f}")
    
    metric_train = test_on_triplets(train_triplet,batch_size=batch_size)
    metric_test = test_on_triplets(val_triplet,batch_size=batch_size)
    
    train_metrics.append(metric_train)
    test_metrics.append(metric_test)
    
    accuracy_train = metric_train[0]
    accuracy_test = metric_test[0]

    print(f"Accuracy on train = {accuracy_train:.5f}")

    print(f"Accuracy on test = {accuracy_test:.5f}")


    
    if save_all or accuracy_test>=max_acc:
        siamese_model.save_weights("siamese_model")
        max_acc = accuracy_test

siamese_model.save_weights("siamese_model-final")

## **Using the Model**


In [ ]:
def extract_encoder(model):
    encoder = get_encoder((256, 256, 3))
    i=0
    for e_layer in model.layers[0].layers[3].layers:
        layer_weight = e_layer.get_weights()
        encoder.layers[i].set_weights(layer_weight)
        i+=1
    return encoder

encoder = extract_encoder(siamese_model)
encoder.save_weights("encoder")
encoder.summary()

## **Classify Images**


In [ ]:
def classify_images(face_list1, face_list2, threshold=1.3):
    # Getting the encodings for the passed faces
    tensor1 = encoder.predict(face_list1)
    tensor2 = encoder.predict(face_list2)
    
    distance = np.sum(np.square(tensor1-tensor2), axis=-1)
    prediction = np.where(distance<=threshold, 0, 1)
    return prediction

In [ ]:
def ModelMetrics(pos_list, neg_list):
    true = np.array([0]*len(pos_list)+[1]*len(neg_list))
    pred = np.append(pos_list, neg_list)
    
    # Compute and print the accuracy
    print(f"\nAccuracy of model: {accuracy_score(true, pred)}\n")
    
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(true, pred)

    categories  = ['Similar','Different']
    names = ['True Similar','False Similar', 'False Different','True Different']
    percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

    labels = [f'{v1}\n{v2}' for v1, v2 in zip(names, percentages)]
    labels = np.asarray(labels).reshape(2,2)

    sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
                xticklabels = categories, yticklabels = categories)

    plt.xlabel("Predicted", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual"   , fontdict = {'size':14}, labelpad = 10)
    plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)


pos_list = np.array([])
neg_list = np.array([])

for data in get_batch(val_triplet, batch_size=256):
    a, p, n = data
    pos_list = np.append(pos_list, classify_images(a, p))
    neg_list = np.append(neg_list, classify_images(a, n))
    

start_time = time.time()
ModelMetrics(pos_list, neg_list)
end_time = time.time()
training_time = end_time - start_time
print(f"\nTotal Validation Time: {training_time:.2f} seconds")

## **Predict Plant & Disease**

In [ ]:
from IPython.display import FileLink, display

# Change to the working directory
%cd /kaggle/working

# Create download links for the files
file1 = FileLink('model_mobile_layers.h5', result_html_prefix="Download: ")
file2 = FileLink('encoder.data-00000-of-00001', result_html_prefix="Download: ")
file3 = FileLink('encoder.index', result_html_prefix="Download: ")
file4 = FileLink('model_densenet.h5', result_html_prefix="Download: ")

# Display the download links
display(file1)
display(file2)
display(file3)
display(file4)


In [12]:
def get_distance(face_list1, face_list2, enc):
    # Getting the encodings for the passed faces
    tensor1 = enc.predict(face_list1)
    tensor2 = enc.predict(face_list2)
    
    distance = np.sum(np.square(tensor1-tensor2), axis=-1)
    return distance

In [13]:
def preprocess_image2(image_path):
    img = load_img(image_path, target_size=(256, 256))
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    return img_array

In [14]:
def predict_plant_type(image_path,model):
    img_array = preprocess_image2(image_path)
    prediction = model.predict(img_array)  # Predict the class probabilities
    predicted_class = np.argmax(prediction, axis=1)  # Get the index of the highest probability
    try:
        plant_type = list(plant_type_mapping.keys())[predicted_class[0]]  # Map index to plant type
    except:
        return "No Info"
    return plant_type

In [15]:
def predict_plant_disease(anchor,plant_type, encoder, max_files = 5) :
    min_distance = float("inf")
    disease = "" 
    best_img = 0
    for i in plant_type_mapping[plant_type]:
        files_num = 0
        distance = 0
        
        for j in train_image_paths[i]:
            image = preprocess_image2(j) 
            distance = get_distance(anchor,image,encoder)
            if distance <min_distance :
                min_distance = distance
                disease = i.split("___")[1]
                best_img = image
            files_num += 1
            if files_num >= max_files:
                break
     
    
    
    return disease, best_img

In [ ]:
test_path = "/kaggle/input/data-vision2/Project Data/Val/images/Cherry_(including_sour)___Powdery_mildew/43f27e79-1bfb-43f7-94d0-ae7f096681a8___FREC_Pwd.M 0254.JPG"
planet_type = predict_plant_type(test_path)

In [ ]:
planet_type

In [ ]:
anchor = preprocess_image2(test_path)
disease, img = predict_plant_disease(anchor, planet_type)

In [ ]:
disease

In [ ]:
image1 = np.squeeze(anchor, axis=0)  # Shape becomes (256, 256, 3)
image2 = np.squeeze(img, axis=0)  # Shape becomes (256, 256, 3)

# Create a figure with two subplots in one row
plt.figure(figsize=(10, 5))

# Display the first image
plt.subplot(1, 2, 1)
plt.imshow(image1)
plt.axis('off')  # Turn off axis
plt.title('Anchor')

# Display the second image
plt.subplot(1, 2, 2)
plt.imshow(image2)
plt.axis('off')  # Turn off axis
plt.title('Prediction')

# Show the images
plt.tight_layout()
plt.show()

## **Prediction** 

In [24]:
 #predict_plant_disease(anchor,plant_type, encoder, max_files = 5) 
def predict_all(test_dir, model, encoder, show_results = True, shots=1):  
    type_acc = 0
    disease_acc = 0
    total = 0
    
    
    for folder_name in os.listdir(test_dir):
        folder_path = os.path.join(test_dir, folder_name)
        
        if os.path.isdir(folder_path):
            for image_name in os.listdir(folder_path):
                expected_plant_type, expected_plant_disease = folder_name.split("___")
                image_path = os.path.join(folder_path, image_name)
                plant_type = predict_plant_type(image_path,model)
                anchor = preprocess_image2(image_path)
                disease, _ = predict_plant_disease(anchor, plant_type,encoder, shots)
                type_acc += (expected_plant_type == plant_type)
                disease_acc += (expected_plant_disease == disease)
                total += 1
                if show_results:
                    print("Expected Plant Type: ", expected_plant_type, "-> Actual Plant Type: ", plant_type)
                    print("Expected Plant Disease: ", expected_plant_disease, " -> Actual Plant Disease: ", disease)
    print("Total Images: " , total)
    print("Predict Planet Type Accuracy: " , type_acc/total)
    print("Predict Planet Disease Accuracy: " , disease_acc/total)


In [ ]:
t = time.time()
predict_all(test_dir, False)
print(f"{int(time.time()-t)} sec)")


In [ ]:
predict_all(test_dir, True)

## **Test Script Part 1**

In [25]:
from tensorflow.keras.models import load_model

loaded_model = load_model('/kaggle/input/last-model/tensorflow2/default/1/Last Model/model_mobile_layers.h5')

In [26]:
input_shape = (256, 256, 3)
loaded_encoder = get_encoder(input_shape)
loaded_encoder.load_weights("/kaggle/input/last-model/tensorflow2/default/1/Last Model/encoder")

In [27]:
path = "/kaggle/input/d/amiratharwat/test-data/Day 2/Test samples for stage 1 (classification) and stage 2 (recognition)/"
predict_all(path,loaded_model,loaded_encoder, True, 1)

Expected Plant Type:  Tomato -> Actual Plant Type:  Tomato
Expected Plant Disease:  Late_blight  -> Actual Plant Disease:  Leaf_Mold
Expected Plant Type:  Tomato -> Actual Plant Type:  Grape
Expected Plant Disease:  Late_blight  -> Actual Plant Disease:  Leaf_blight_(Isariopsis_Leaf_Spot)
Expected Plant Type:  Tomato -> Actual Plant Type:  Tomato
Expected Plant Disease:  Late_blight  -> Actual Plant Disease:  Late_blight
Expected Plant Type:  Tomato -> Actual Plant Type:  Tomato
Expected Plant Disease:  Late_blight  -> Actual Plant Disease:  Late_blight
Expected Plant Type:  Tomato -> Actual Plant Type:  Tomato
Expected Plant Disease:  Late_blight  -> Actual Plant Disease:  Tomato_Yellow_Leaf_Curl_Virus
Expected Plant Type:  Tomato -> Actual Plant Type:  Apple
Expected Plant Disease:  Late_blight  -> Actual Plant Disease:  Apple_scab
Expected Plant Type:  Tomato -> Actual Plant Type:  Tomato
Expected Plant Disease:  Late_blight  -> Actual Plant Disease:  Late_blight
Expected Plant Type

In [35]:
#TEST SCRIPT 2 PART 2
def predict_anchor(test_dir, enc):
    anchor_image = "Anchor.JPG"
    anchor_path = os.path.join(test_dir,anchor_image)
    anchor_img = preprocess_image2(anchor_path)

    min_distance = float("inf")
    for image_name in os.listdir(test_dir):
        if anchor_image == image_name:
            continue
        image_path = os.path.join(test_dir, image_name)
        img = preprocess_image2(image_path)
        distance = get_distance(anchor_img, img, enc)
        
        print("image = ", image_name)
        print("distance = ", distance)
        
        if distance < min_distance:
            min_distance = distance
            best_img = image_name
    print(best_img)

In [36]:
path2 = "/kaggle/input/d/amiratharwat/test-data/Day 2/One-few shot recognition part 2/"
# Anchor = orange_1.jpg
predict_anchor(path2,loaded_encoder)

image =  Squash_powdery_mildew.JPG
distance =  [1.9708558]
image =  Sobyean_healthy.JPG
distance =  [1.8630794]
image =  Potato_late_blight.JPG
distance =  [1.9973358]
image =  Blueberry_healthy.JPG
distance =  [0.25614953]
image =  Tomato_healthy.JPG
distance =  [1.6404233]
image =  Rasperry_healthy.JPG
distance =  [1.949973]
image =  Apple_scab.JPG
distance =  [0.18690091]
image =  Blueberry_healthy2.JPG
distance =  [0.11734532]
image =  Orange_citrus_greening.JPG
distance =  [1.8746457]
image =  Philodendron.png
distance =  [1.9965302]
image =  Peach_Bacterial_spot.JPG
distance =  [1.9782144]
image =  Hibisicus_plant_gudhal.png
distance =  [1.980779]
Blueberry_healthy2.JPG


In [ ]:
import time

# Keep the notebook running
while True:
    print("Notebook is running...")  # Optional: Log to indicate the notebook is alive
    time.sleep(600)  # Sleep for 10 minutes (adjust as needed)
